In [1]:
import json
import pandas as pd
from os import listdir
import re
from unidecode import unidecode
from tqdm import tqdm
from pre_process_functions import *

In [ ]:
### Load Clean Topics
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/00-prep_data/topics_speeches.json') as f:
    clean_topics_data = json.load(f)

In [ ]:
mapping_topics = {}
for i in clean_topics_data:
    mapping_topics[i['id_speech']] = i['clean_topic']

In [ ]:
### Load Specches Disaggregated
with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/00-prep_data/legislatura64_speeches.json') as f:
    speech = json.load(f)

In [ ]:
for i in tqdm(speech):
    i['topic_speech'] = mapping_topics[i['id_speech']]
    i['clean_speech'] = last_cleaning(pre_processText(i['text_speech']))

In [ ]:
set(' '.join(pd.DataFrame(speech).clean_speech.unique()))

In [ ]:
#with open('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/01-clean_data/disaaggregated_data/legislatura64_disaggregated.json', 'w') as f:
#    json.dump(speech, f)

In [ ]:
### Clean Topics For LDA

In [ ]:
def last_cleaning(text):
    clean = re.sub('\t', '', text)
    clean = re.sub(r"\b\d+|\d+\b", "", clean)
    clean = re.sub('ó', 'o', clean)
    clean = re.sub('í', 'i', clean)
    clean = re.sub('[-]', '', clean)
    clean = re.sub('[.]', '', clean)
    clean = remove_accents(clean)
    return(clean)

In [2]:
all_legis = []

In [3]:
path_speeches = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Congressional-dataMX/data/02-outcomes/01-policy_positioning/01-text_analysis/01-clean_data/disaaggregated_data/'
for i in listdir(path_speeches):
    with open(path_speeches + i) as f:
        temp = json.load(f)
        for h in temp:
            del h['link_speech']
            del h['text_speech']
            del h['clean_speech']
            all_legis.append(h)

In [18]:
def remove_specific_words(text):
    clean_text = []
    for i in text.split(' '):
        if i in ['a', 'le', 'y']:
            continue
        else:
            clean_text.append(i)
    return(' '.join(clean_text))

In [19]:
for i in tqdm(all_legis):
    i['clean_topic'] = last_cleaning(pre_processText(i['topic_speech']))
    i['clean_topic'] = last_cleaning(i['clean_topic'])
    i['clean_topic'] = remove_specific_words(i['clean_topic'])

100%|██████████| 40415/40415 [20:33<00:00, 32.75it/s] 


In [20]:
words = ' '.join(pd.DataFrame(all_legis).clean_topic.unique()).split(' ')
words.sort()

In [21]:
unique_words = []
for i in words:
    if i not in unique_words:
        unique_words.append(i)

In [26]:
len(pd.DataFrame(all_legis).clean_topic.unique()), len(pd.DataFrame(all_legis).clean_topic)

(4790, 40415)

In [29]:
pd.DataFrame(all_legis)[pd.DataFrame(all_legis)['clean_topic'].isna()]

,legislatura,year_session,year_speech,month_speech,day_speech,topic_speech,inc_party,inc_name,id_speech,id,clean_topic


In [33]:
df = pd.DataFrame(all_legis)

In [35]:
df.to_csv('../../../../data/02-outcomes/01-policy_positioning/01-text_analysis/01-clean_data/topics/topics_speeches.csv', encoding = 'utf-8')